<a href="https://www.kaggle.com/code/akshaypatil7015/used-car-price-prediction-main?scriptVersionId=174509098" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/used-cars-price-prediction/test-data.csv
/kaggle/input/used-cars-price-prediction/train-data.csv


# Data Loading

In [2]:
df=pd.read_csv('/kaggle/input/used-cars-price-prediction/train-data.csv')
df.head()

,Unnamed: 0,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price
0,0,Maruti Wagon R LXI CNG,Mumbai,2010,72000,CNG,Manual,First,26.6 km/kg,998 CC,58.16 bhp,5.0,NaN,1.75
1,1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67 kmpl,1582 CC,126.2 bhp,5.0,NaN,12.50
2,2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,18.2 kmpl,1199 CC,88.7 bhp,5.0,8.61 Lakh,4.50
3,3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77 kmpl,1248 CC,88.76 bhp,7.0,NaN,6.00
4,4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.2 kmpl,1968 CC,140.8 bhp,5.0,NaN,17.74


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6019 entries, 0 to 6018
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         6019 non-null   int64  
 1   Name               6019 non-null   object 
 2   Location           6019 non-null   object 
 3   Year               6019 non-null   int64  
 4   Kilometers_Driven  6019 non-null   int64  
 5   Fuel_Type          6019 non-null   object 
 6   Transmission       6019 non-null   object 
 7   Owner_Type         6019 non-null   object 
 8   Mileage            6017 non-null   object 
 9   Engine             5983 non-null   object 
 10  Power              5983 non-null   object 
 11  Seats              5977 non-null   float64
 12  New_Price          824 non-null    object 
 13  Price              6019 non-null   float64
dtypes: float64(2), int64(3), object(9)
memory usage: 658.5+ KB


In [4]:
df.describe()

,Unnamed: 0,Year,Kilometers_Driven,Seats,Price
count,6019.000000,6019.000000,6.019000e+03,5977.000000,6019.000000
mean,3009.000000,2013.358199,5.873838e+04,5.278735,9.479468
std,1737.679967,3.269742,9.126884e+04,0.808840,11.187917
min,0.000000,1998.000000,1.710000e+02,0.000000,0.440000
25%,1504.500000,2011.000000,3.400000e+04,5.000000,3.500000
50%,3009.000000,2014.000000,5.300000e+04,5.000000,5.640000
75%,4513.500000,2016.000000,7.300000e+04,5.000000,9.950000
max,6018.000000,2019.000000,6.500000e+06,10.000000,160.000000


In [5]:
name=df['Name'].str.split(n=3, expand=True)
df['company']=name[0]
df['model']=name[1]

In [6]:
df.drop(['Unnamed: 0','New_Price','Location','Name'], axis=1, inplace=True)
df.dropna(inplace=True)
df.columns = list(map(str.lower, df.columns))
df.head()

,year,kilometers_driven,fuel_type,transmission,owner_type,mileage,engine,power,seats,price,company,model
0,2010,72000,CNG,Manual,First,26.6 km/kg,998 CC,58.16 bhp,5.0,1.75,Maruti,Wagon
1,2015,41000,Diesel,Manual,First,19.67 kmpl,1582 CC,126.2 bhp,5.0,12.50,Hyundai,Creta
2,2011,46000,Petrol,Manual,First,18.2 kmpl,1199 CC,88.7 bhp,5.0,4.50,Honda,Jazz
3,2012,87000,Diesel,Manual,First,20.77 kmpl,1248 CC,88.76 bhp,7.0,6.00,Maruti,Ertiga
4,2013,40670,Diesel,Automatic,Second,15.2 kmpl,1968 CC,140.8 bhp,5.0,17.74,Audi,A4


In [7]:
print(f'unique values in fuel_type is {df.fuel_type.nunique()}')
print(f'unique values in transmission is {df.transmission.nunique()}')
print(f'unique values in owner_type is {df.owner_type.nunique()}')
print(f'unique values in company is {df.company.nunique()}')
print(f'unique values in model is {df.model.nunique()}')

unique values in fuel_type is 4
unique values in transmission is 2
unique values in owner_type is 4
unique values in company is 31
unique values in model is 209


In [8]:
df.isna().sum()

year                 0
kilometers_driven    0
fuel_type            0
transmission         0
owner_type           0
mileage              0
engine               0
power                0
seats                0
price                0
company              0
model                0
dtype: int64

In [9]:
df['mileage'] = df['mileage'].str.replace('km/kg', '')
df['mileage'] = df['mileage'].str.replace('kmpl', '')
df['engine'] = df['engine'].str.replace('CC', '')
df['power'] = df['power'].str.replace('bhp', '')

df['mileage'] = pd.to_numeric(df['mileage'], errors='coerce')
df['engine'] = pd.to_numeric(df['engine'], errors='coerce')
df['power'] = pd.to_numeric(df['power'], errors='coerce')

In [10]:
df.head()

,year,kilometers_driven,fuel_type,transmission,owner_type,mileage,engine,power,seats,price,company,model
0,2010,72000,CNG,Manual,First,26.60,998,58.16,5.0,1.75,Maruti,Wagon
1,2015,41000,Diesel,Manual,First,19.67,1582,126.20,5.0,12.50,Hyundai,Creta
2,2011,46000,Petrol,Manual,First,18.20,1199,88.70,5.0,4.50,Honda,Jazz
3,2012,87000,Diesel,Manual,First,20.77,1248,88.76,7.0,6.00,Maruti,Ertiga
4,2013,40670,Diesel,Automatic,Second,15.20,1968,140.80,5.0,17.74,Audi,A4


In [11]:
df['power'].fillna(df['power'].mean(), inplace=True)

df.isnull().sum()

/tmp/ipykernel_18/4098723069.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['power'].fillna(df['power'].mean(), inplace=True)


year                 0
kilometers_driven    0
fuel_type            0
transmission         0
owner_type           0
mileage              0
engine               0
power                0
seats                0
price                0
company              0
model                0
dtype: int64

In [12]:
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
le = LabelEncoder()

df['fuel_type']=le.fit_transform(df['fuel_type'])
df['transmission']=le.fit_transform(df['transmission'])

ordinal_map = {'First':1, 'Second':2, 'Third':3, 'Fourth & Avove':4}
df['owner_type']=df['owner_type'].map(ordinal_map)
df.dropna(inplace=True)

In [13]:
def lowerdata(df, column):
    return df[column].str.lower()

df['company']=lowerdata(df, 'company')
df['model']=lowerdata(df, 'model')

In [14]:
df.head()

,year,kilometers_driven,fuel_type,transmission,owner_type,mileage,engine,power,seats,price,company,model
0,2010,72000,0,1,1.0,26.60,998,58.16,5.0,1.75,maruti,wagon
1,2015,41000,1,1,1.0,19.67,1582,126.20,5.0,12.50,hyundai,creta
2,2011,46000,3,1,1.0,18.20,1199,88.70,5.0,4.50,honda,jazz
3,2012,87000,1,1,1.0,20.77,1248,88.76,7.0,6.00,maruti,ertiga
4,2013,40670,1,0,2.0,15.20,1968,140.80,5.0,17.74,audi,a4


In [15]:
company_model_names=df[['company','model']]
print(f'company_model_names_shape {company_model_names.shape}')
company_model_names.head()

company_model_names_shape (5967, 2)


,company,model
0,maruti,wagon
1,hyundai,creta
2,honda,jazz
3,maruti,ertiga
4,audi,a4


In [16]:
company_model_names['c_name']=le.fit_transform(company_model_names['company'])
company_model_names['m_name']=le.fit_transform(company_model_names['model'])
company_model_names.head()

/tmp/ipykernel_18/2597518510.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  company_model_names['c_name']=le.fit_transform(company_model_names['company'])
/tmp/ipykernel_18/2597518510.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  company_model_names['m_name']=le.fit_transform(company_model_names['model'])


,company,model,c_name,m_name
0,maruti,wagon,17,186
1,hyundai,creta,10,53
2,honda,jazz,9,98
3,maruti,ertiga,17,67
4,audi,a4,1,10


In [17]:
# created dict for maping company and models

company_name_list=company_model_names['company'].tolist()
company_label_list=company_model_names['c_name'].tolist()

model_name_list=company_model_names['model'].tolist()
model_label_list=company_model_names['m_name'].tolist()


company_dict=dict(zip(company_name_list, company_label_list))
model_dict=dict(zip(model_name_list, model_label_list))

df['company']=df['company'].map(company_dict)
df['model']=df['model'].map(model_dict)

In [18]:
df.head()

,year,kilometers_driven,fuel_type,transmission,owner_type,mileage,engine,power,seats,price,company,model
0,2010,72000,0,1,1.0,26.60,998,58.16,5.0,1.75,17,186
1,2015,41000,1,1,1.0,19.67,1582,126.20,5.0,12.50,10,53
2,2011,46000,3,1,1.0,18.20,1199,88.70,5.0,4.50,9,98
3,2012,87000,1,1,1.0,20.77,1248,88.76,7.0,6.00,17,67
4,2013,40670,1,0,2.0,15.20,1968,140.80,5.0,17.74,1,10


## PCA

In [19]:
X=df.drop('price', axis=1)
y=df[['price']]
print(X.shape, y.shape)

(5967, 11) (5967, 1)


In [20]:
from sklearn.preprocessing import StandardScaler
std_scaler = StandardScaler()
X_standardized = std_scaler.fit_transform(X)

In [21]:
from sklearn.decomposition import PCA
pca = PCA(n_components=5, random_state=21)
X_pca = pca.fit_transform(X_standardized)
X_pca_df=pd.DataFrame(X_pca)
X_pca_df.rename(columns={0:'pca_1', 1: 'pca_2', 2: 'pca_3', 3: 'pca_4', 4: 'pca_5'}, inplace=True)
X_pca_df

,pca_1,pca_2,pca_3,pca_4,pca_5
0,-1.781379,1.500096,-0.771200,1.417025,1.110270
1,-0.227191,-0.617608,-0.830765,0.767772,-0.580500
2,-1.125805,-0.560925,0.890829,-0.351973,-0.177099
3,-0.417359,1.448177,-0.534984,0.737013,-1.730690
4,1.777652,-1.953434,0.949432,1.375359,-0.623322
...,...,...,...,...,...
5962,-1.872273,0.755012,-1.395655,0.718173,0.860062
5963,-1.573551,0.704590,-1.107722,1.052008,1.097641
5964,1.965765,3.323359,0.563547,-0.078945,-1.279675
5965,-1.671570,0.603321,0.296417,-1.030022,0.923142


# Model Building

In [22]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 31)

In [23]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(4773, 11) (1194, 11) (4773, 1) (1194, 1)


In [24]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
lin_mod = LinearRegression()
ran_mod = RandomForestRegressor()

In [25]:
lin_mod.fit(X_train, y_train)
ran_mod.fit(X_train, y_train)

lin_pred = lin_mod.predict(X_test)
ran_pred = ran_mod.predict(X_test)

/tmp/ipykernel_18/794179974.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  ran_mod.fit(X_train, y_train)


In [26]:
from sklearn.metrics import mean_squared_error, r2_score
from math import sqrt

lin_rmse = sqrt(mean_squared_error(lin_pred, y_test))
lin_r2 = r2_score(lin_pred, y_test)

ran_rmse = sqrt(mean_squared_error(ran_pred, y_test))
ran_r2 = r2_score(ran_pred, y_test)

## After PCA

In [27]:
X_train_pca, X_test_pca, y_train_pca, y_test_pca = train_test_split(X_pca_df, y, test_size = 0.2, random_state = 41)

lin_mod_pca = LinearRegression()
ran_mod_pca = RandomForestRegressor()

In [28]:
lin_mod_pca.fit(X_train_pca, y_train_pca)
ran_mod_pca.fit(X_train_pca, y_train_pca)

lin_pred_pca = lin_mod_pca.predict(X_test_pca)
ran_pred_pca = ran_mod_pca.predict(X_test_pca)


lin_rmse_pca = sqrt(mean_squared_error(lin_pred_pca, y_test_pca))
lin_r2_pca = r2_score(lin_pred_pca, y_test_pca)

ran_rmse_pca = sqrt(mean_squared_error(ran_pred_pca, y_test_pca))
ran_r2_pca = r2_score(ran_pred_pca, y_test_pca)

/tmp/ipykernel_18/960291428.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  ran_mod_pca.fit(X_train_pca, y_train_pca)


In [29]:
print(f'Before PCA linear model r2 = {lin_r2} and RMSE = {lin_rmse}')
print(f'After PCA linear model r2 = {lin_r2_pca} and RMSE = {lin_rmse_pca}')
print()
print(f'Before PCA random forest model r2 = {ran_r2} and RMSE = {ran_rmse}')
print(f'After PCA random forest model r2 = {ran_r2_pca} and RMSE = {ran_rmse_pca}')

Before PCA linear model r2 = 0.3947754204745004 and RMSE = 8.006684325151921
After PCA linear model r2 = 0.498531663405019 and RMSE = 6.619665326804102

Before PCA random forest model r2 = 0.8749108919679693 and RMSE = 3.816733844366044
After PCA random forest model r2 = 0.8281456422099877 and RMSE = 4.262150930915965


In [30]:
from sklearn.metrics import get_scorer_names
get_scorer=get_scorer_names()
get_scorer

['accuracy',
 'adjusted_mutual_info_score',
 'adjusted_rand_score',
 'average_precision',
 'balanced_accuracy',
 'completeness_score',
 'explained_variance',
 'f1',
 'f1_macro',
 'f1_micro',
 'f1_samples',
 'f1_weighted',
 'fowlkes_mallows_score',
 'homogeneity_score',
 'jaccard',
 'jaccard_macro',
 'jaccard_micro',
 'jaccard_samples',
 'jaccard_weighted',
 'matthews_corrcoef',
 'max_error',
 'mutual_info_score',
 'neg_brier_score',
 'neg_log_loss',
 'neg_mean_absolute_error',
 'neg_mean_absolute_percentage_error',
 'neg_mean_gamma_deviance',
 'neg_mean_poisson_deviance',
 'neg_mean_squared_error',
 'neg_mean_squared_log_error',
 'neg_median_absolute_error',
 'neg_negative_likelihood_ratio',
 'neg_root_mean_squared_error',
 'normalized_mutual_info_score',
 'positive_likelihood_ratio',
 'precision',
 'precision_macro',
 'precision_micro',
 'precision_samples',
 'precision_weighted',
 'r2',
 'rand_score',
 'recall',
 'recall_macro',
 'recall_micro',
 'recall_samples',
 'recall_weighted',

In [31]:
from sklearn.model_selection import cross_val_score

lin_cv_score = cross_val_score(lin_mod, X_train, y_train, cv=7, scoring='r2')
ran_cv_score = cross_val_score(ran_mod, X_train, y_train, cv=7, scoring='r2')

lin_cv_score_pca = cross_val_score(lin_mod_pca, X_train_pca, y_train_pca, cv=7, scoring='r2')
ran_cv_score_pca = cross_val_score(ran_mod_pca, X_train_pca, y_train_pca, cv=7, scoring='r2')

/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed whe

In [32]:
data={'model':['lin_cv_score', 'ran_cv_score', 'lin_cv_score_pca', 'ran_cv_score_pca'], 
      'r2_score':[lin_cv_score.mean(), ran_cv_score.mean(), lin_cv_score_pca.mean(),ran_cv_score_pca.mean()]}

In [33]:
df_cross_val=pd.DataFrame(data)
df_cross_val

,model,r2_score
0,lin_cv_score,0.700570
1,ran_cv_score,0.881285
2,lin_cv_score_pca,0.641551
3,ran_cv_score_pca,0.847955


random forest regression R2_score was 0.8887. Therefore selected model was Random forest regressor

In [34]:
# model fit on over all dataset
ran_mod=ran_mod.fit(X,y)

/tmp/ipykernel_18/3914132145.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  ran_mod=ran_mod.fit(X,y)


# Preparing unseen data

In [35]:
df_test=pd.read_csv('/kaggle/input/used-cars-price-prediction/test-data.csv')

In [36]:
df_test.head()

,Unnamed: 0,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price
0,0,Maruti Alto K10 LXI CNG,Delhi,2014,40929,CNG,Manual,First,32.26 km/kg,998 CC,58.2 bhp,4.0,NaN
1,1,Maruti Alto 800 2016-2019 LXI,Coimbatore,2013,54493,Petrol,Manual,Second,24.7 kmpl,796 CC,47.3 bhp,5.0,NaN
2,2,Toyota Innova Crysta Touring Sport 2.4 MT,Mumbai,2017,34000,Diesel,Manual,First,13.68 kmpl,2393 CC,147.8 bhp,7.0,25.27 Lakh
3,3,Toyota Etios Liva GD,Hyderabad,2012,139000,Diesel,Manual,First,23.59 kmpl,1364 CC,null bhp,5.0,NaN
4,4,Hyundai i20 Magna,Mumbai,2014,29000,Petrol,Manual,First,18.5 kmpl,1197 CC,82.85 bhp,5.0,NaN


In [37]:
name_test=df_test['Name'].str.split(n=3, expand=True)
df_test['company']=name_test[0]
df_test['model']=name_test[1]

In [38]:
df_test.drop(['Unnamed: 0','New_Price','Location','Name'], axis=1, inplace=True)
df_test.dropna(inplace=True)
df_test.head()

,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,company,model
0,2014,40929,CNG,Manual,First,32.26 km/kg,998 CC,58.2 bhp,4.0,Maruti,Alto
1,2013,54493,Petrol,Manual,Second,24.7 kmpl,796 CC,47.3 bhp,5.0,Maruti,Alto
2,2017,34000,Diesel,Manual,First,13.68 kmpl,2393 CC,147.8 bhp,7.0,Toyota,Innova
3,2012,139000,Diesel,Manual,First,23.59 kmpl,1364 CC,null bhp,5.0,Toyota,Etios
4,2014,29000,Petrol,Manual,First,18.5 kmpl,1197 CC,82.85 bhp,5.0,Hyundai,i20


In [39]:
df_test.columns = list(map(str.lower, df_test.columns))
df_test.head()

,year,kilometers_driven,fuel_type,transmission,owner_type,mileage,engine,power,seats,company,model
0,2014,40929,CNG,Manual,First,32.26 km/kg,998 CC,58.2 bhp,4.0,Maruti,Alto
1,2013,54493,Petrol,Manual,Second,24.7 kmpl,796 CC,47.3 bhp,5.0,Maruti,Alto
2,2017,34000,Diesel,Manual,First,13.68 kmpl,2393 CC,147.8 bhp,7.0,Toyota,Innova
3,2012,139000,Diesel,Manual,First,23.59 kmpl,1364 CC,null bhp,5.0,Toyota,Etios
4,2014,29000,Petrol,Manual,First,18.5 kmpl,1197 CC,82.85 bhp,5.0,Hyundai,i20


In [40]:
df_test['mileage'] = df_test['mileage'].str.replace('km/kg', '')
df_test['mileage'] = df_test['mileage'].str.replace('kmpl', '')
df_test['engine'] = df_test['engine'].str.replace('CC', '')
df_test['power'] = df_test['power'].str.replace('bhp', '')

df_test['mileage'] = pd.to_numeric(df_test['mileage'], errors='coerce')
df_test['engine'] = pd.to_numeric(df_test['engine'], errors='coerce')
df_test['power'] = pd.to_numeric(df_test['power'], errors='coerce')
df_test.head()

,year,kilometers_driven,fuel_type,transmission,owner_type,mileage,engine,power,seats,company,model
0,2014,40929,CNG,Manual,First,32.26,998,58.20,4.0,Maruti,Alto
1,2013,54493,Petrol,Manual,Second,24.70,796,47.30,5.0,Maruti,Alto
2,2017,34000,Diesel,Manual,First,13.68,2393,147.80,7.0,Toyota,Innova
3,2012,139000,Diesel,Manual,First,23.59,1364,NaN,5.0,Toyota,Etios
4,2014,29000,Petrol,Manual,First,18.50,1197,82.85,5.0,Hyundai,i20


In [41]:
df_test['power'].fillna(df_test['power'].mean(), inplace=True)
df_test.isna().sum()

/tmp/ipykernel_18/345844466.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_test['power'].fillna(df_test['power'].mean(), inplace=True)


year                 0
kilometers_driven    0
fuel_type            0
transmission         0
owner_type           0
mileage              0
engine               0
power                0
seats                0
company              0
model                0
dtype: int64

In [42]:
df_test['company']=lowerdata(df_test, 'company')
df_test['model']=lowerdata(df_test, 'model')
df_test.head()

,year,kilometers_driven,fuel_type,transmission,owner_type,mileage,engine,power,seats,company,model
0,2014,40929,CNG,Manual,First,32.26,998,58.200000,4.0,maruti,alto
1,2013,54493,Petrol,Manual,Second,24.70,796,47.300000,5.0,maruti,alto
2,2017,34000,Diesel,Manual,First,13.68,2393,147.800000,7.0,toyota,innova
3,2012,139000,Diesel,Manual,First,23.59,1364,110.398472,5.0,toyota,etios
4,2014,29000,Petrol,Manual,First,18.50,1197,82.850000,5.0,hyundai,i20


In [43]:
df_test.shape

(1223, 11)

In [44]:
df_test.head()

,year,kilometers_driven,fuel_type,transmission,owner_type,mileage,engine,power,seats,company,model
0,2014,40929,CNG,Manual,First,32.26,998,58.200000,4.0,maruti,alto
1,2013,54493,Petrol,Manual,Second,24.70,796,47.300000,5.0,maruti,alto
2,2017,34000,Diesel,Manual,First,13.68,2393,147.800000,7.0,toyota,innova
3,2012,139000,Diesel,Manual,First,23.59,1364,110.398472,5.0,toyota,etios
4,2014,29000,Petrol,Manual,First,18.50,1197,82.850000,5.0,hyundai,i20


In [45]:
df_test['fuel_type']=le.fit_transform(df_test['fuel_type'])
df_test['transmission']=le.fit_transform(df_test['transmission'])

ordinal_map = {'First':1, 'Second':2, 'Third':3, 'Fourth & Avove':4}
df_test['owner_type']=df_test['owner_type'].map(ordinal_map)

df_test['company']=df_test['company'].map(company_dict)
df_test['model']=df_test['model'].map(model_dict)

df_test.dropna(inplace=True)

df_test.head()

,year,kilometers_driven,fuel_type,transmission,owner_type,mileage,engine,power,seats,company,model
0,2014,40929,0,1,1.0,32.26,998,58.200000,4.0,17.0,16.0
1,2013,54493,3,1,2.0,24.70,796,47.300000,5.0,17.0,16.0
2,2017,34000,1,1,1.0,13.68,2393,147.800000,7.0,27.0,97.0
3,2012,139000,1,1,1.0,23.59,1364,110.398472,5.0,27.0,69.0
4,2014,29000,3,1,1.0,18.50,1197,82.850000,5.0,10.0,92.0


In [46]:
final_pred=ran_mod.predict(df_test)

In [47]:
final_pred

array([ 3.1598    ,  2.2914    , 17.4295    , ...,  2.847     ,
        4.4156    , 19.46896667])